<a href="https://colab.research.google.com/github/AbhinayPoloju/Torch-Audio-Course/blob/main/HFAudio_Unit7_Hands_onexercise_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
!pip install --upgrade --quiet pip
!pip install --quiet git+https://github.com/huggingface/transformers.git
!pip install --upgrade accelerate
!pip install gradio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from huggingface_hub import notebook_login
import torch
from transformers import pipeline
from datasets import load_dataset
from IPython.display import Audio
from transformers import BarkModel
from transformers import AutoProcessor
import numpy as np

In [4]:
!nvidia-smi

Tue May  7 04:14:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
notebook_login()

# Speech translation

In [8]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
#device="cpu"
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-large-v2", device=device
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:


dataset = load_dataset("facebook/voxpopuli", "en", split="validation", streaming=True)
sample = next(iter(dataset))
print(sample)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for facebook/voxpopuli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/voxpopuli
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


{'audio_id': '20160526-0900-PLENARY-3-en_20160526-10:18:50_2', 'language': 0, 'audio': {'path': 'dev_part_0/20160526-0900-PLENARY-3-en_20160526-10:18:50_2.wav', 'array': array([-0.00088501, -0.0007019 , -0.00057983, ...,  0.00085449,
        0.00067139,  0.00061035]), 'sampling_rate': 16000}, 'raw_text': 'Many measures have been taken, not only in September but also in March, and of course we see some effects of those measures perhaps not enough, but there are effects of those measures, and the situation could have been worse if we did not have taken those measures.', 'normalized_text': 'many measures have been taken not only in september but also in march and of course we see some effects of those measures perhaps not enough but there are effects of those measures and the situation could have been worse if we did not have taken those measures.', 'gender': 'male', 'speaker_id': 'None', 'is_gold_transcript': True, 'accent': 'None'}


In [10]:
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [11]:
def translate(audio):
    outputs = pipe(audio, max_new_tokens=256, generate_kwargs={"task": "transcribe","language":"chinese"})
    return outputs["text"]

In [12]:
translate(sample["audio"].copy())

'很多措施在9月和3月中都被採取。當然,我們看到一些措施的影響,也許不足夠,但這些措施的影響確實存在,如果沒有採取這些措施,情況可能會更糟糕。'

# Text-to-speech

In [13]:
model = BarkModel.from_pretrained("suno/bark-small")
processor = AutoProcessor.from_pretrained("suno/bark")

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [14]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [15]:
def synthesise(text_prompt,voice_preset="v2/zh_speaker_1"):
    inputs = processor(text_prompt, voice_preset=voice_preset)
    speech_output = model.generate(**inputs.to(device))
    synthesised_rate = model.generation_config.sample_rate
    return synthesised_rate,speech_output

In [16]:
target_dtype = np.int16
max_range = np.iinfo(target_dtype).max


def speech_to_speech_translation(audio,voice_preset="v2/zh_speaker_1"):
    translated_text = translate(audio)
    synthesised_rate,synthesised_speech = synthesise(translated_text,voice_preset)
    synthesised_speech = (synthesised_speech.cpu().numpy() * max_range).astype(np.int16)
    return synthesised_rate,synthesised_speech

In [17]:
synthesised_rate,synthesised_speech = speech_to_speech_translation(sample["audio"],"v2/zh_speaker_1")

Audio(synthesised_speech, rate=synthesised_rate)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [ ]:
import gradio as gr

demo = gr.Blocks()

mic_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(sources="microphone", type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy"),
)

file_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(sources="upload", type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy"),
)

with demo:
    gr.TabbedInterface([mic_translate, file_translate], ["Microphone", "Audio File"])

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b2e716c17fefab443c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 270, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1847, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1433, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    r